In [42]:
import os
import pandas as pd
import string
from datetime import datetime
from os import path
import json
import numpy as np

In [43]:
with open(path.expanduser("~") + "/sys_specific.json") as file: 
    params = json.load(file)
_HOME_ = params["home"] + "/PHD"

excel_path = _HOME_ + "/MSFD/Data/Modulo1/"

In [44]:
with open(_HOME_ + "/ISPRA_20152017_Analysis/params.json") as file: 
    params = json.load(file)
best_path = params["best_path"]
best_path_sard = params["best_path_sard"]
ordered_regions = params["ordered_regions"]
seasons = params["seasons"]
sorted_season = params["sorted_season"]
sea_index = params["sea_index"]
ordered_id = params["ordered_id"]
sea_index_array = np.array(list(sea_index.values()))
del params

# import data

In [45]:
raw_data = pd.read_csv(excel_path + "/chemical_fis_raw.csv", dtype = str)

In [46]:
raw_data.rename(columns= {"NationalStationID" : "id", "Determinand_Nutrients" : "Nutrient"}, inplace=True)

# check id and date

In [20]:
raw_data["id"] = raw_data["id"].apply(lambda x: x.split(".")[0])
raw_data["id"] = raw_data["id"].apply(lambda x: str.strip(x))
raw_data["id"] = raw_data["id"].apply(lambda x: x.replace("M000200", "MS00200"))

In [21]:
raw_data["Year"] = raw_data["Year"].apply(lambda x: x.split(".")[0])
raw_data["Month"] = raw_data["Month"].apply(lambda x: x.split(".")[0])
raw_data["Day"] = raw_data["Day"].apply(lambda x: x.split(".")[0])

In [22]:
raw_data["Date"] = pd.to_datetime([f"{year}-{month}-{day}" for year, month, day in zip(raw_data["Year"], raw_data["Month"], raw_data["Day"])], format = "%Y-%m-%d")

# check nutrients

In [28]:
#ox saturation
raw_data.loc[raw_data["Nutrient"].isin(['Temperature (water)', 'Temperature ', "Temperature"]), "Nutrient"] = "T"
raw_data.loc[raw_data["Nutrient"].isin(['Electrical Conductivity', ' Electrical Conductivity', 'Electrical conductivity']), "Nutrient"] = 'E_cond'
raw_data.loc[raw_data["Nutrient"].isin(['Dissolved Oxygen', 'Dissolved oxygen']), "Nutrient"] = 'DO'
raw_data.loc[raw_data["Nutrient"].isin([' ', 'Oxygen Saturation', 'Oxygen saturation']), "Nutrient"] = 'O_sat'

In [29]:
raw_data.loc[raw_data["Nutrient"].isin([' pH', 'pH ']), "Nutrient"] = 'pH'
raw_data.loc[raw_data["Nutrient"].isin(['Chlorophyll a']), "Nutrient"] = 'Chla'
raw_data.loc[raw_data["Nutrient"].isin(['Secchi depth']), "Nutrient"] = 'Secchi_depth'
raw_data.loc[raw_data["Nutrient"].isin(['Nitrate']), "Nutrient"] = 'NO3'
raw_data.loc[raw_data["Nutrient"].isin(['Nitrite']), "Nutrient"] = 'NO2'
raw_data.loc[raw_data["Nutrient"].isin(['Ammonium']), "Nutrient"] = 'NH4'
raw_data.loc[raw_data["Nutrient"].isin(['Total phosphorus', 'Total Phosphorus']), "Nutrient"] = 'TP'
raw_data.loc[raw_data["Nutrient"].isin(['Orthophosphates']), "Nutrient"] = 'PO4'
raw_data.loc[raw_data["Nutrient"].isin(['Total nitrogen', 'Total Nitrogen']), "Nutrient"] = 'TN'
raw_data.loc[raw_data["Nutrient"].isin(['Silicate']), "Nutrient"] = 'SiO4'

## make wide table

In [32]:
raw_data.drop(columns = ["Year", "Month", "Day", "Time"], inplace=True)

In [41]:
raw_data.loc[(~raw_data["Remarks"].isna()) & (raw_data["Nutrient"] != "Chla"), :].to_csv(_HOME_ + "/ISPRA_20152017_analysis/Create_dataset/remarks_nut.csv", index=0)